In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-
import os
import gc

import numpy as np
import torch
import torch.backends.cudnn as cudnn
from torch import nn

from dataloader import svhn
from models import ShuffleNetV2
from src import freeze_influence, hessians, selection

device = "cuda" if torch.cuda.is_available() else "cpu"
target_removal_label = 0

In [3]:
def load_net(net, path, _assert=True):
    if _assert:
        assert os.path.isfile(path), f"Error: no checkpoint file found! {path}"
    
    if not os.path.isfile(path):
        return None
    checkpoint = torch.load(path)
    net.load_state_dict(checkpoint["net"])
    return net


def save_net(net, path):
    dir, filename = os.path.split(path)
    if not os.path.isdir(dir):
        os.makedirs(dir)

    state = {
        "net": net.state_dict(),
    }
    torch.save(state, path)


def test(net, dataloader, criterion, label, include):
    net.eval()
    with torch.no_grad():
        net_loss = 0
        correct = 0
        total = 0
        num_data = 0
        for _, (inputs, targets) in enumerate(dataloader):
            if include:
                idx = targets == label
            else:
                idx = targets != label
            inputs = inputs[idx]
            targets = targets[idx]
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            net_loss += loss * len(inputs)
            num_data +=  len(inputs)

            total += targets.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()

        accuracy = correct / total * 100
        net_loss /= num_data
        return net_loss, accuracy

def influence_test(net, dataloader, criterion, target_label):
    def sample_test(net, criterion, inputs, targets):
        net.eval()
        with torch.no_grad():
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            _, predicted = outputs.max(1)
            correct = predicted.eq(targets).sum().item()

            return loss, correct

    self_loss = 0
    self_correct = 0
    num_self_inputs = 0
    
    exclusive_loss = 0
    exclusive_correct = 0
    num_exclusive_inputs = 0
    
    for _, (inputs, targets) in enumerate(dataloader):
        target_idx = (targets == target_label)
        batch_self_loss, batch_self_correct = sample_test(net, criterion, inputs[target_idx], targets[target_idx])
        batch_exclusive_loss, batch_exclusive_correct = sample_test(net, criterion, inputs[~target_idx], targets[~target_idx])
        
        len_self_batch = len(inputs[target_idx])
        self_loss += batch_self_loss * len_self_batch
        self_correct += batch_self_correct
        num_self_inputs += len_self_batch
        
        len_exclusive_batch = len(inputs[~target_idx])
        exclusive_loss += batch_exclusive_loss * len_exclusive_batch
        exclusive_correct += batch_exclusive_correct
        num_exclusive_inputs += len_exclusive_batch
        
    self_loss /= num_self_inputs
    self_acc = self_correct / num_self_inputs * 100
    exclusive_loss /= num_exclusive_inputs
    exclusive_acc = exclusive_correct / num_exclusive_inputs * 100
    
    return self_loss, self_acc, exclusive_loss, exclusive_acc

def projected_influence(net, total_loss, target_loss, index_list, tol, step, max_iter, verbose):
    num_param = sum(p.numel() for p in net.parameters() if p.requires_grad)
    full_param_index_list = np.arange(num_param)
    influence = hessians.generalized_influence(
        net, total_loss, target_loss, full_param_index_list, tol=tol, step=step, max_iter=max_iter, verbose=verbose
    )
    return influence[idx]

def f1_score(self_acc, test_acc):
    self_acc /= 100
    test_acc /= 100
    if self_acc == 1 and test_acc == 0:
        return 0
    return 2 * (1 - self_acc) * test_acc / (1 - self_acc + test_acc)

In [4]:
net = ShuffleNetV2().to(device)
net_name = "ShuffleNetV2"

if device == "cuda":
    cudnn.benchmark = True

net_path = f"checkpoints/tab1/{net_name}/cross_entropy/ckpt_0.0.pth"
net = load_net(net, net_path)

net.eval()
num_param = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(
    f"==> Building {net_name} finished. "
    + f"\n    Number of parameters: {num_param}"
)

criterion = nn.CrossEntropyLoss()

# Data
print("==> Preparing data..")
batch_size = 512
num_workers = 16
num_sample_batch = 1
num_target_sample = 512

data_loader = svhn.SVHNDataLoader(batch_size, num_workers, validation=False)
train_loader, test_loader = data_loader.get_data_loaders()

loss, original_acc = test(net, test_loader, criterion, 11, False)
print(
    f"Original loss and acc : {loss:.4f}, {original_acc:.2f}%"
)

==> Building ShuffleNetV2 finished. 
    Number of parameters: 1263854
==> Preparing data..
Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat
Original loss and acc : 0.1826, 95.24%


In [5]:
verbose = False
num_exp = 30

removal_inputs = list()
removal_targets = list()
for batch_idx, (inputs, targets) in enumerate(train_loader):
    idx = targets == target_removal_label
    removal_inputs.append(inputs[idx])
    removal_targets.append(targets[idx])
removal_inputs = torch.cat(removal_inputs)
removal_targets = torch.cat(removal_targets)

ratio_list = np.arange(2, 21, 3) / 100
ratio_list = np.append(ratio_list, np.arange(30, 101, 10) / 100) 
result_list_LowestKOutputs = []
result_list_LowestKGradients = []
result_list_HighestKOutputs = []
result_list_HighestKGradients = []
result_list_Random = []
tol = 1e-9

parser_list = [selection.LowestKOutputs,
              selection.LowestKGradients,
              selection.HighestKOutputs,
              selection.HighestKGradients,
              selection.Random,]

In [ ]:
for i in range(num_exp):
    print(f"==================Iteration {i}==================")
    sample_idx = np.random.choice(len(removal_inputs), num_target_sample, replace=False)
    for param_ratio in ratio_list:
        for parser in parser_list:

            print(f"{parser.__name__} - ratio: {int(param_ratio*100)}%")
            # Initialize network
            net = load_net(net, net_path)

            # Compute total loss
            total_loss = 0
            for batch_idx, (inputs, targets) in enumerate(test_loader):
                if batch_idx >= num_sample_batch:
                    break
                idx = targets != target_removal_label
                inputs, targets = inputs[idx], targets[idx]
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                total_loss += criterion(outputs, targets)

            # Sampling the target removal data
            sample_removal_inputs = removal_inputs[sample_idx]
            sample_removal_targets = removal_targets[sample_idx]
            
            # Make hooks
            net_parser = parser(net, param_ratio)
            net_parser.register_hooks()

            if isinstance(net_parser, selection.HighestKGradients) or \
               isinstance(net_parser, selection.LowestKGradients):
                # Compute target loss
                target_loss = (
                    criterion(net(sample_removal_inputs.to(device)), sample_removal_targets.to(device))
                    * len(removal_inputs)
                    / (len(train_loader.dataset) - len(removal_inputs))
                )
                target_loss.backward()
                net_parser.remove_hooks()

            target_loss = (
                criterion(net(sample_removal_inputs.to(device)), sample_removal_targets.to(device))
                * len(removal_inputs)
                / (len(train_loader.dataset) - len(removal_inputs))
            )
            
            # Delete hooks
            index_list = net_parser.get_parameters()
            net_parser.remove_hooks()
            
            influence = hessians.generalized_influence(
                net, total_loss, target_loss, index_list, tol=tol, step=7, max_iter=200, verbose=verbose
            )
            del target_loss, total_loss
            gc.collect()
            torch.cuda.empty_cache()
            
            influence = influence * 0.06 / torch.norm(influence)
                
            scale = 1
            score = 0
            best_score = -1
            count = 1
            save_path = (
                f"checkpoints/tab1/{net_name}/{net_parser.__class__.__name__}/{param_ratio}_{i}.pth"
            )
            while True:
                net_parser.update_network(influence * scale)

                self_loss, self_acc, exclusive_loss, exclusive_acc = influence_test(net, test_loader, 
                                                                                    criterion, target_removal_label)
                score = f1_score(self_acc, exclusive_acc)
                
                if best_score < score and self_acc < 0.5:
                    best_result = [exclusive_acc, exclusive_loss, self_acc, self_loss, score]
                    best_score = score
                    save_net(net, save_path)
                    
                if verbose:
                    print(
                    f"{count} - test acc: {exclusive_acc:2.2f}, test loss: {exclusive_loss:.4f} |" +
                    f" self-acc: {self_acc:2.2f}%, self loss: {self_loss:.4f} | score: {score:.7f}",
                    end='\r'
                    )

                if self_acc < 0.1 or count >= 200:
                    if parser == selection.LowestKOutputs:
                        result_list_LowestKOutputs += best_result
                    elif parser == selection.LowestKGradients:
                        result_list_LowestKGradients += best_result
                    elif parser == selection.HighestKOutputs:
                        result_list_HighestKOutputs += best_result
                    elif parser == selection.HighestKGradients:
                        result_list_HighestKGradients += best_result
                    else:
                        result_list_Random += best_result
                    
                    print(f"test acc: {best_result[0]:2.2f}, test loss: {best_result[1]:.4f} | " +
                          f"self-acc: {best_result[2]:2.2f}%, self loss: {best_result[3]:.4f} | " +
                          f"Score: {best_result[4]:.7f}" + " " * 20) 
                    break
                elif count >= 20 and best_score < 0.2:
                    scale = 5
                elif count >= 50 and best_score < 0.5:
                    print(f"test acc: {best_result[0]:2.2f}, test loss: {best_result[1]:.4f} | " +
                          f"self-acc: {best_result[2]:2.2f}%, self loss: {best_result[3]:.4f} | " +
                          f"Score: {best_result[4]:.7f}" + " " * 20) 
                    save_net(net, save_path)
                    break

                count += 1
                
        print("")

==================Iteration 0==================
LowestKOutputs - ratio: 2%
test acc: 88.05, test loss: 0.4344 | self-acc: 0.11%, self loss: 4.4774 | Score: 0.9359363                    
LowestKGradients - ratio: 2%


In [ ]:
num_exp = 30
mean_exclusive_acc_list = np.zeros((len(parser_list), len(ratio_list[:-1])))
var_exclusive_acc_list = np.zeros((len(parser_list), len(ratio_list[:-1])))

for j, param_ratio in enumerate(ratio_list):
    print("")
    for i, parser in enumerate(parser_list):
        print(f"{parser.__name__} - ratio: {int(param_ratio*100)}%")
        
        self_loss_list = np.empty(0)
        self_acc_list = np.empty(0)
        exclusive_loss_list = np.empty(0)
        exclusive_acc_list = np.empty(0)
        f1_score_list = np.empty(0)

        for exp_idx in range(num_exp):

            load_path = (
                f"checkpoints/tab1/{net_name}/{parser.__name__}/{param_ratio}_{exp_idx}.pth"
            )
            net = ShuffleNetV2().to(device)
            net = load_net(net, load_path, False)
            if net is None:
                continue
            self_loss, self_acc, exclusive_loss, exclusive_acc = influence_test(net, test_loader, 
                                                                    criterion, target_removal_label)
            
            score = f1_score(self_acc, exclusive_acc)
            
            self_loss_list = np.append(self_loss_list, self_loss.detach().cpu().numpy())
            self_acc_list = np.append(self_acc_list, self_acc)
            exclusive_loss_list = np.append(exclusive_loss_list, exclusive_loss.detach().cpu().numpy())
            exclusive_acc_list = np.append(exclusive_acc_list, exclusive_acc)
            f1_score_list = np.append(f1_score_list, score)
            print(
            f"  test acc: {exclusive_acc:2.2f}, test loss: {exclusive_loss:.4f} | " +
            f"self-acc: {self_acc:2.2f}%, self loss: {self_loss:.4f} | score: {score:.4f}",
            end='\r'
            )
            
        mean_self_loss = np.mean(self_loss_list)
        mean_self_acc = np.mean(self_acc_list)
        mean_exclusive_loss = np.mean(exclusive_loss_list)
        mean_exclusive_acc = np.mean(exclusive_acc_list)
        mean_f1_score = np.mean(f1_score_list)
                
        var_self_loss = np.var(self_loss_list)**.5
        var_self_acc = np.var(self_acc_list)**.5
        var_exclusive_loss = np.var(exclusive_loss_list)**.5
        var_exclusive_acc = np.var(exclusive_acc_list)**.5
        var_f1_score = np.var(f1_score_list)**.5
            
        mean_exclusive_acc_list[i, j] = mean_exclusive_acc
        var_exclusive_acc_list[i, j] = var_exclusive_acc
        
        
        print(
        f"{mean_exclusive_acc:2.2f} $\pm$ {var_exclusive_acc:2.2f} & " +
        f"{mean_exclusive_loss:.2f} $\pm$ {var_exclusive_loss:.2f} & ", end=""
        )
        print(
        f"{mean_self_acc:2.2f} $\pm$ {var_self_acc:2.2f} & " +
        f"{mean_self_loss:.2f} $\pm$ {var_self_loss:.2f} & {mean_f1_score:.4f} \\\\" + " " * 10
        )

In [ ]:
parser = selection.LowestKOutputs
param_ratio = 1.0

exclusive_acc_list = np.empty(0)
for exp_idx in range(num_exp):
    print(f"exp: {exp_idx}", end="\r")
    load_path = (
        f"checkpoints/tab1/{net_name}/{parser.__name__}/{param_ratio}_{exp_idx}.pth"
    )
    net = ShuffleNetV2().to(device)
    net = load_net(net, load_path)
    if net is None:
        continue
    self_loss, self_acc, exclusive_loss, exclusive_acc = influence_test(net, test_loader, 
                                                            criterion, target_removal_label)

    score = f1_score(self_acc, exclusive_acc)
    exclusive_acc_list = np.append(exclusive_acc_list, exclusive_acc)
            
plain_influence_acc = np.mean(exclusive_acc_list)

In [ ]:
import matplotlib.pyplot as plt

# Colors and markers
colors = ['#090446', '#786F52', '#F71735', '#FEB95F', '#000000', "#ED3D15", "#777777"]  # Example hex codes
colors = ['#48C2FF', '#004DC9', '#ACCFE8', '#FFAD76', '#EF300A', "#000000", "#CCCCCC"]  # Example hex codes
markers = ['o', 's', '^', 'D', 'p']  # Example marker styles

# Fontsize variable
fontsize = 20

# Abbreviation for brevity
pr_wo1 = ratio_list[:-1]*100
meal = mean_exclusive_acc_list
veal = var_exclusive_acc_list

# Creating the plots
plt.figure(figsize=(10, 7))
plt.plot(pr_wo1, meal[2,:], marker=markers[2], color=colors[0], label='Highest-k outputs', linewidth=2)
plt.plot(pr_wo1, meal[3,:], marker=markers[3], color=colors[1], label='Highest-k gradients', linewidth=2)
plt.plot(pr_wo1, meal[4,:], marker=markers[4], color=colors[2], label='Random', linewidth=2)
plt.plot(pr_wo1, meal[0,:], marker=markers[0], color=colors[3], label='Lowest-k outputs', linewidth=2)
plt.plot(pr_wo1, meal[1,:], marker=markers[1], color=colors[4], label='Lowest-k gradients', linewidth=2)

#
# plt.fill_between(pr_wo1, meal[0,:]-veal[0,:], meal[0,:]+veal[0,:], color=colors[3], alpha=0.2)
# plt.fill_between(pr_wo1, meal[1,:]-veal[1,:], meal[1,:]+veal[1,:], color=colors[4], alpha=0.2)
# plt.fill_between(pr_wo1, meal[2,:]-veal[2,:], meal[2,:]+veal[2,:], color=colors[0], alpha=0.2)
# plt.fill_between(pr_wo1, meal[3,:]-veal[3,:], meal[3,:]+veal[3,:], color=colors[1], alpha=0.2)
# plt.fill_between(pr_wo1, meal[4,:]-veal[4,:], meal[4,:]+veal[4,:], color=colors[2], alpha=0.2)


# Horizontal values
plt.axhline(y=original_acc, color=colors[5], linestyle='-.', dashes=(5, 5, 1, 5), linewidth=2)
plt.fill_between(pr_wo1, original_acc*0.98, original_acc*1.02, color="yellow", alpha=0.3)
plt.axhline(y=plain_influence_acc, color=colors[6], linestyle='--', dashes=(5, 3), label="Naive influence function")

# Adding legend and labels with specified fontsize
plt.xlabel('Modification ratio (%)', fontsize=fontsize+2)
plt.ylabel('Test accuracy (%)', fontsize=fontsize+2)
# plt.legend(fontsize=fontsize-1, ncol=2, columnspacing=0.9)

# Adding ticks
plt.xticks(pr_wo1, fontsize=fontsize)
plt.xlim((np.min(pr_wo1), np.max(pr_wo1)))
plt.yticks(range(0, 101, 20), fontsize=fontsize)
plt.ylim((20, 110))

plt.grid(True, linewidth=0.25, color="#BBBBBB")

# Display the plot
plt.savefig('test_accuracy.pdf', format='pdf', bbox_inches='tight', transparent=True)
plt.show()